In [5]:
import ee
import geemap
import geopandas as gpd

In [2]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
# Read the shapefile using geopandas
shapefile_path = r'D:\Raul Perez\scripts2023\Gee_Arauco\data\r20_oficial.shp'
gdf = gpd.read_file(shapefile_path)

In [7]:
gdf['geometry'] = gdf['geometry'].simplify(tolerance=10)

In [9]:
#for download files from araucaria update 05_Araucaria2gee

predios = geemap.geopandas_to_ee(gdf)
Map.addLayer(predios, {}, 'Countries')

Could not convert the geojson to ee.Geometry()


Exception: Invalid GeoJSON geometry.

In [4]:
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

In [5]:
style = {'description_width': 'initial'}
uploader = widgets.FileUpload(
    description='Upload data',
    accept='.zip, .json, .geojson',
    multiple=False,
    button_style='primary',
    style=style,
)

submit = widgets.Button(
    description='Display data', button_style='success', tooltip='Click me', style=style
)

reset = widgets.Button(
    description='Reset', button_style='warning', tooltip='Click me', style=style
)

In [6]:
with output_widget:
    print('Upload shapefile or \ngeojson as a zip file')
    display(uploader)
    display(submit)
    display(reset)

In [ ]:
def get_vector(upload_widget, out_dir=None):

    import zipfile
    import glob

    if out_dir is None:
        out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    vector = None

    try:
        [uploaded_file] = upload_widget.value
        file = upload_widget.value[uploaded_file]
        name = file['metadata']['name']
        content = file['content']
        out_file = os.path.join(out_dir, name)
        with open(out_file, "wb") as fp:
            fp.write(content)

        if name.endswith('.zip'):
            with zipfile.ZipFile(out_file, "r") as zip_ref:
                extract_dir = os.path.join(
                    out_dir, name[:-4] + "_" + geemap.random_string(3)
                )
                zip_ref.extractall(extract_dir)
                files = glob.glob(extract_dir + '/*.shp')
                if len(files) > 0:
                    shp = files[0]
                    vector = geemap.shp_to_ee(shp)
                else:
                    files = glob.glob(extract_dir + '/*.geojson')
                    if len(files) > 0:
                        geojson = files[0]
                        vector = geemap.geojson_to_ee(geojson)
        else:
            vector = geemap.geojson_to_ee(out_file)

    except Exception as e:
        print(e)

    return vector